In [ ]:
from oneatlas import OneAtlasClient
import json
import geopandas as gpd

# Process overview
- Convert input aoi layer into geojson polygons
- For each aoi polygon feature:
  - Search API for available images
  - Plot the quicklook for each image found for that feature
  - Place an order for the chosen image
  - Download the order once completed

In [ ]:
# function to convert polygon gdf to bounding box list
aoi_gdf = gpd.read_file("inputs/test_quarries.shp")

In [ ]:
# If necessary buffer the image
aoi_gdf["geometry"] = aoi_gdf.buffer(1000)

In [ ]:
def aoi_gdf_to_search_geojson(aoi_gdf, uid_column=None):
    """Convert geodataframe to geojson ensuring unique ids per feature"""


    if uid_column is None:


        uid_column = "id"


        aoi_gdf[uid_column] = aoi_gdf.index


    else:


        if not aoi_gdf[uid_column].is_unique:


            raise ValueError(
                f"Values in '{uid_column}' are not unique. Consider using default uid_column=None to add uids."
            )


    print(
        f"uid_column '{uid_column}' values {', '.join(str(i) for i in aoi_gdf[uid_column].tolist())}"
    )


    json_str = aoi_gdf[[uid_column, "geometry"]].to_crs(epsg=4326).to_json(drop_id=True)


    return json.loads(json_str), aoi_gdf[uid_column].tolist()

In [ ]:
search_geojson, id_vals = aoi_gdf_to_search_geojson(aoi_gdf)

In [ ]:
def get_feature_by_id(geojson, id_vals, search_id, uid_column="id"):
    """Filter geojson features to just one id"""


    if search_id not in id_vals:


        raise ValueError(f"id {search_id} not in list of id values")


    for feature in geojson["features"]:


        if feature["properties"][uid_column] == search_id:
            return feature


    return None

In [ ]:
# search_test = search_geojson["features"][0]["geometry"]
SEARCH_ID = 0
search_feature = get_feature_by_id(search_geojson, id_vals, SEARCH_ID)

In [ ]:
# Read local config.json to get api key
with open("config.json", "r") as file:
    config = json.load(file)
api_key = config["api_key"]

client = OneAtlasClient(api_key=api_key)

In [ ]:
results = client.search(
    {
        "cloudCover": "[0,30]",
        "incidenceAngle": "[0,40]",
        "processingLevel": "SENSOR",
        "relation": "contains",
        "geometry": search_feature["geometry"],
        "constellation": "PHR",
    }
)
client.extract_results(results)

In [ ]:
client.show_result()

See the order options [here](https://www.geoapi-airbusds.com/guides/oneatlas-data/g-order-product/)

In [ ]:
order_body = {
    "kind": "order.data.product",
    "products": [
        {
            "productType": "multiSpectral",
            "radiometricProcessing": "REFLECTANCE",
            "imageFormat": "image/geotiff",
            "crsCode": "urn:ogc:def:crs:EPSG::4326",
            "id": client.current_image,
            "aoi": search_feature["geometry"],
        }
    ],
}
price = client.get_price(order_body)
client.get_price(order_body)["price"]

In [ ]:
client.create_order(order_body)

In [ ]:
orders = client.list_orders(kind="order.data.product", status="delivered")

In [ ]:
order = orders["items"][0]
client.download_order_to_file(order, "outputs/order.zip")